<a href="https://colab.research.google.com/github/VictorManuelOrt/Biometric/blob/main/Ranking_Biometrics_DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opencv-python numpy


In [ ]:
!pip install onnx onnxruntime pydantic==1.10.16 huggingface-hub pyyaml
!pip install --no-deps open-iris

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.1/151.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.12.3
    Uninstalling pydantic-2.12.3:
      Successfully uninstalled pydantic-2.12.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langsmith 0.6.4 requires pydantic<3,>=2, but you have pydantic 1.10.16 which is incompatible.
google-genai 1.55.0 requires pydantic<3.0.0,>=2.9.0, but you have pydantic 1.10.16 which is i

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("olankadhim/multimodal-biometric-dataset-mulb")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'multimodal-biometric-dataset-mulb' dataset.
Path to dataset files: /kaggle/input/multimodal-biometric-dataset-mulb


In [ ]:
import iris
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import os
from PIL import Image
import matplotlib.pyplot as plt
import os
import math
import pandas as pd

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
path

'/kaggle/input/multimodal-biometric-dataset-mulb'

In [ ]:
def gabor_kernel(theta, phi, sigma):
    """
    Construye un kernel de Gabor complejo:

    g(x,y) = exp(-(x^2 + y^2)/sigma^2) *
             exp(2*pi*i*theta*(x*cos(phi) + y*sin(phi)))

    El tamaño del kernel se fija como +/- 3*sigma.
    """

    # Tamaño del kernel (impar)
    half = int(np.ceil(3 * sigma))
    K = 2 * half + 1

    # Sistema de coordenadas centrado en (0,0)
    x = np.arange(-half, half + 1)
    y = np.arange(-half, half + 1)
    X, Y = np.meshgrid(x, y)

    # Proyección direccional
    s = X * np.cos(phi) + Y * np.sin(phi)

    # Gaussiana (ventana suave)
    gaussian = np.exp(-(X**2 + Y**2) / sigma**2)

    # Onda compleja (frecuencia + orientación)
    wave = np.exp(2j * np.pi * theta * s)

    # Kernel de Gabor complejo
    kernel = gaussian * wave

    return kernel

def apply_gabor(image, kernel):
    """
    Aplica un filtro de Gabor complejo a una imagen.
    Retorna la magnitud de la respuesta.
    """

    # Asegurar tipo float
    image = image.astype(np.float64)

    # Separar partes real e imaginaria del kernel
    real_k = np.real(kernel)
    imag_k = np.imag(kernel)

    # Convolución (modo 'same' mantiene tamaño)
    real_resp = cv.filter2D(image, cv.CV_64F, real_k)
    imag_resp = cv.filter2D(image, cv.CV_64F, imag_k)

    # Magnitud (energía local)
    magnitude = np.sqrt(real_resp**2 + imag_resp**2)

    return magnitude

In [ ]:
def distancia_euclidiana(dic1, dic2, clave1, clave2):
    v1 = np.array(dic1[clave1])
    v2 = np.array(dic2[clave2])
    return np.linalg.norm(v1 - v2)


In [ ]:
# -----------------------------
# Parámetros fijos
# -----------------------------
sigma = 80.0
kernel_size = int(2 * np.ceil(3 * sigma) + 1)

# 3 frecuencias espaciales
thetas = [0.02, 0.04, 0.08, 0.16, 0.32]

# 4 orientaciones: k*pi/4
phis = [k * np.pi / 4 for k in range(4)]


In [ ]:
sigma

80.0

In [ ]:
#lo siguiente seria juntar todo en uns funcion y  solo ajustar
#en una celda parametrso del filstro de gabor
#nombre de imagen a sacar caracteristicas
#guardar valores en vectores para hacer comparaciones


#umbral de pertenencia en terminos con juegos?

In [ ]:
Features_Dictionary = {}

In [ ]:
def Add_Features_Dictionary(name_image):
    """
    Genera un vector
    """

    name_img=name_image
    img_pixels = cv.imread('/kaggle/input/multimodal-biometric-dataset-mulb/MULB dataset/iris dataset/'+name_img, cv.IMREAD_GRAYSCALE)
    if img_pixels is None:
      return

    iris_pipeline = iris.IRISPipeline(env=iris.IRISPipeline.DEBUGGING_ENVIRONMENT)

    output = iris_pipeline(iris.IRImage(img_data=img_pixels, image_id="image_id", eye_side="right"))
    iris_visualizer = iris.visualisation.IRISVisualizer()


    out = iris_visualizer.plot_normalized_iris(normalized_iris=iris_pipeline.call_trace['normalization'],)

    plt.savefig("iris_normalized.png", dpi=300, bbox_inches="tight")
    plt.close()

    Features_Vector = []
    # Leer imagen en escala de grises
    img = cv.imread("iris_normalized.png", cv.IMREAD_GRAYSCALE)
    img = img.astype(np.float64) / 255.0
    # Visualización
    fig, axes = plt.subplots(
        len(thetas),
        len(phis),
        figsize=(12, 8),
        sharex=True,
        sharey=True
    )
    for i, theta in enumerate(thetas):
        for j, phi in enumerate(phis):
            # Construir kernel
            kernel = gabor_kernel(theta, phi, sigma)
            # Aplicar filtro
            response = apply_gabor(img, kernel)
            # Normalizar respuesta
            response_norm = response / (response.max() + 1e-8)
            ax = axes[i, j]
            ax.imshow(response_norm, cmap='gray')
            ax.axis('off')
            # Títulos solo en bordes (más limpio)
            ax.set_title(rf"$\theta = {theta},\ \varphi = {j}\pi/4,\mu_c = {round(response.mean(),3)},\sigma_c = {round(response.std(),3)} $", fontsize=9)
            Features_Vector.append(response.mean())
            Features_Vector.append(response.std())
    plt.suptitle(
        rf"Banco de filtros de Gabor (3 frecuencias × 4 orientaciones){name_img}",
        fontsize=14
    )

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig(name_img.replace("/", "_"))
    plt.close()

    # Guardamos las caracteristicas en el diccionario
    Features_Dictionary[name_image] = Features_Vector



In [ ]:
mulb_dataset_path = os.path.join(path, 'MULB dataset')
hand_or_iris="iris dataset"
hand_dataset_path = os.path.join(mulb_dataset_path, hand_or_iris)
!ls -F "{hand_dataset_path}"
!ls -F "{os.path.join(hand_dataset_path, 'AW')}"

A/   AL/  AX/  BI/  BU/  CF/  CR/  DC/	DO/  E/   EL/  EX/  FI/  FU/  GF/  S/
AA/  AM/  AY/  BJ/  BV/  CG/  CS/  DD/	DP/  EA/  EM/  EY/  FJ/  FV/  H/   T/
AB/  AN/  AZ/  BK/  BW/  CH/  CT/  DE/	DQ/  EB/  EN/  EZ/  FK/  FW/  I/   U/
AC/  AO/  B/   BL/  BX/  CI/  CU/  DF/	DR/  EC/  EO/  F/   FL/  FX/  J/   V/
AD/  AP/  BA/  BM/  BY/  CJ/  CV/  DG/	DS/  ED/  EP/  FA/  FM/  FY/  K/   W/
AE/  AQ/  BB/  BN/  BZ/  CK/  CW/  DH/	DT/  EE/  EQ/  FB/  FN/  FZ/  L/   X/
AF/  AR/  BC/  BO/  C/	 CL/  CX/  DI/	DU/  EF/  ER/  FC/  FO/  G/   M/   Y/
AG/  AS/  BD/  BP/  CA/  CM/  CY/  DJ/	DV/  EG/  ES/  FD/  FP/  GA/  N/   Z/
AH/  AT/  BE/  BQ/  CB/  CN/  CZ/  DK/	DW/  EH/  ET/  FE/  FQ/  GB/  O/
AI/  AU/  BF/  BR/  CC/  CO/  D/   DL/	DX/  EI/  EU/  FF/  FR/  GC/  P/
AJ/  AV/  BG/  BS/  CD/  CP/  DA/  DM/	DY/  EJ/  EV/  FG/  FS/  GD/  Q/
AK/  AW/  BH/  BT/  CE/  CQ/  DB/  DN/	DZ/  EK/  EW/  FH/  FT/  GE/  R/
10.jpg	12.jpg	14.jpg	16.jpg	18.jpg	1.jpg	2.jpg  4.jpg  6.jpg  8.jpg
11.jpg	13.jpg	15.jpg	17.jpg	1

In [ ]:
name_img="CF/5.jpg"
Add_Features_Dictionary(name_img)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


iris_semseg_upp_scse_mobilenetv2.onnx:   0%|          | 0.00/56.1M [00:00<?, ?B/s]

In [ ]:
Features_Dictionary={}

In [ ]:
claves=['R/18.jpg',
 'R/17.jpg',
 'S/17.jpg',
 'S/12.jpg',
 'EL/12.jpg',
 'EL/2.jpg',
 'AL/8.jpg',
 'AL/4.jpg',
 'BR/4.jpg',
 'BR/12.jpg',
 'A/12.jpg',
 'A/8.jpg',
 'BI/4.jpg',
 'BI/17.jpg',
 'CF/11.jpg',
 'CF/5.jpg']

In [ ]:
for name_img in claves:
  Add_Features_Dictionary(name_img)

In [ ]:
claves = list(Features_Dictionary.keys())
claves

['R/18.jpg',
 'R/17.jpg',
 'S/17.jpg',
 'S/12.jpg',
 'EL/12.jpg',
 'EL/2.jpg',
 'AL/8.jpg',
 'AL/4.jpg',
 'BR/4.jpg',
 'BR/12.jpg',
 'A/12.jpg',
 'A/8.jpg',
 'BI/4.jpg',
 'BI/17.jpg',
 'CF/11.jpg',
 'CF/5.jpg']

In [ ]:
Features_Dictionary

{'R/18.jpg': [np.float64(99.4673416201159),
  np.float64(99.23556425681822),
  np.float64(39.91164717497144),
  np.float64(29.833040536286052),
  np.float64(217.19998145207524),
  np.float64(158.27160820347873),
  np.float64(47.53556837870368),
  np.float64(47.840831066710564),
  np.float64(61.035146380166125),
  np.float64(79.97531387079316),
  np.float64(19.69502707495512),
  np.float64(18.36504935266955),
  np.float64(167.61072608096674),
  np.float64(149.42909420185734),
  np.float64(18.80695604290823),
  np.float64(21.092661060617477),
  np.float64(50.59596792166374),
  np.float64(103.69745762999088),
  np.float64(7.306834866581133),
  np.float64(6.843289758138623),
  np.float64(169.87673823527226),
  np.float64(161.10653787494434),
  np.float64(5.979204709958978),
  np.float64(5.789573933484571),
  np.float64(20.513235028451167),
  np.float64(41.94309013496088),
  np.float64(2.582828247945255),
  np.float64(1.7916072951058792),
  np.float64(72.65420285313112),
  np.float64(69.608

In [ ]:
Test_Dictionary={}

In [ ]:
def Add_Test_Dictionary(name_image):
    """
    Genera un vector
    """

    name_img=name_image
    img_pixels = cv.imread('/kaggle/input/multimodal-biometric-dataset-mulb/MULB dataset/iris dataset/'+name_img, cv.IMREAD_GRAYSCALE)
    if img_pixels is None:
      return

    iris_pipeline = iris.IRISPipeline(env=iris.IRISPipeline.DEBUGGING_ENVIRONMENT)

    output = iris_pipeline(iris.IRImage(img_data=img_pixels, image_id="image_id", eye_side="right"))
    iris_visualizer = iris.visualisation.IRISVisualizer()


    out = iris_visualizer.plot_normalized_iris(normalized_iris=iris_pipeline.call_trace['normalization'],)

    plt.savefig("iris_normalized.png", dpi=300, bbox_inches="tight")
    plt.close()

    Features_Vector = []
    # Leer imagen en escala de grises
    img = cv.imread("iris_normalized.png", cv.IMREAD_GRAYSCALE)
    img = img.astype(np.float64) / 255.0
    # Visualización
    fig, axes = plt.subplots(
        len(thetas),
        len(phis),
        figsize=(12, 8),
        sharex=True,
        sharey=True
    )
    for i, theta in enumerate(thetas):
        for j, phi in enumerate(phis):
            # Construir kernel
            kernel = gabor_kernel(theta, phi, sigma)
            # Aplicar filtro
            response = apply_gabor(img, kernel)
            # Normalizar respuesta
            response_norm = response / (response.max() + 1e-8)
            ax = axes[i, j]
            ax.imshow(response_norm, cmap='gray')
            ax.axis('off')
            # Títulos solo en bordes (más limpio)
            ax.set_title(rf"$\theta = {theta},\ \varphi = {j}\pi/4,\mu_c = {round(response.mean(),3)},\sigma_c = {round(response.std(),3)} $", fontsize=9)
            Features_Vector.append(response.mean())
            Features_Vector.append(response.std())
    plt.suptitle(
        rf"Banco de filtros de Gabor (3 frecuencias × 4 orientaciones){name_img}",
        fontsize=14
    )

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig(name_img.replace("/", "_"))
    plt.close()

    # Guardamos las caracteristicas en el diccionario
    Test_Dictionary[name_image] = Features_Vector


In [ ]:

claves_test=['R/15.jpg',
 'S/14.jpg',
 'EL/6.jpg',
 'AL/7.jpg',
 'BR/10.jpg', 'AW/10.jpg']


In [ ]:
#Add_Test_Dictionary('AW/10.jpg')

In [ ]:
for name_img in claves_test:
  Add_Test_Dictionary(name_img)

In [ ]:
Test_Dictionary

{'R/15.jpg': [np.float64(116.11988130581146),
  np.float64(102.4477123340748),
  np.float64(54.90712334127434),
  np.float64(40.31346441934172),
  np.float64(236.42324647805415),
  np.float64(156.80884732423448),
  np.float64(58.90629412202324),
  np.float64(60.339171548090405),
  np.float64(74.47496843443459),
  np.float64(81.54390839157628),
  np.float64(24.77855234751191),
  np.float64(17.868986562948145),
  np.float64(180.13545080211813),
  np.float64(155.75442480589422),
  np.float64(24.7710864650757),
  np.float64(19.814246587458207),
  np.float64(55.02178351763667),
  np.float64(100.71087260021505),
  np.float64(10.321933696776256),
  np.float64(6.6599136985386),
  np.float64(170.0195973171411),
  np.float64(161.16974493119952),
  np.float64(8.253998400733261),
  np.float64(4.846649309157055),
  np.float64(21.873535450165775),
  np.float64(40.02639270717725),
  np.float64(2.4838740254633302),
  np.float64(1.6101285328696844),
  np.float64(71.16102114223631),
  np.float64(67.9039

In [ ]:

df = pd.DataFrame(index=claves, columns=claves_test)

for i in claves:
    for j in claves_test:
        df.loc[i, j] = distancia_euclidiana(Features_Dictionary, Test_Dictionary, i, j)

for col in df.columns:
    orden = df[col].sort_values(ascending=True)
    print(f"Columna {col}:")
    print(orden)
    print()
coincidencias = {}

for col in df.columns:
    # ordenamos la columna ascendente y obtenemos los índices
    indices_ordenados = df[col].sort_values(ascending=True).index

    # tomamos los dos primeros caracteres de la columna
    prefix_col = col[:2]

    # tomamos los dos primeros índices de la columna ordenada
    primeros_indices = indices_ordenados[:3]

    # contamos cuántos coinciden en los dos primeros caracteres
    count = sum(1 for idx in primeros_indices if idx[:2] == prefix_col)

    coincidencias[col] = count

print(coincidencias)



Columna R/15.jpg:
R/18.jpg       42.19083
R/17.jpg      43.650214
CF/5.jpg      44.984263
S/12.jpg       51.04118
AL/4.jpg      58.406778
A/8.jpg       66.651446
S/17.jpg      67.260996
BR/4.jpg      72.228146
CF/11.jpg     73.811633
AL/8.jpg      73.962858
BR/12.jpg     75.449994
A/12.jpg      76.475289
BI/4.jpg      82.030561
EL/2.jpg       86.37763
BI/17.jpg     88.346406
EL/12.jpg    110.558687
Name: R/15.jpg, dtype: object

Columna S/14.jpg:
S/12.jpg      49.135498
S/17.jpg      52.573934
AL/4.jpg      53.837241
BI/4.jpg      69.475289
BI/17.jpg      71.92925
CF/5.jpg      75.569813
BR/12.jpg     77.649545
R/18.jpg      78.014017
AL/8.jpg      79.751237
A/12.jpg      79.927572
R/17.jpg      80.455097
BR/4.jpg      83.699601
A/8.jpg       89.970077
CF/11.jpg    100.894719
EL/12.jpg    114.324559
EL/2.jpg     118.542708
Name: S/14.jpg, dtype: object

Columna EL/6.jpg:
EL/2.jpg      35.166157
CF/11.jpg     39.761179
A/8.jpg       40.927574
R/18.jpg      59.962988
BR/4.jpg      65.755

In [ ]:
valores_dos = sum(1 for v in coincidencias.values() if v >= 2)
print(valores_dos)

4


In [ ]:
len(coincidencias)

6



```
# Lo siguiente es el ranking con disancia educlidiana
```



Referencias:
- Tahmasebi, A., & Pourghassem, H. (2017). Robust intra-class distance-based approach for multimodal biometric game theory-based rank-level fusion of ear, palmprint and signature. Iranian Journal of Science and Technology, Transactions of Electrical Engineering, 41(1), 51-64.
- Worldcoin AI. (2023). IRIS: Iris recognition inference system of the Worldcoin project [Computer software]. https://github.com/worldcoin/open-iris